<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [4]:
! pip install gensim

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Busque los textos de HP Lovercraft acá

https://www.kaggle.com/datasets/bennijesus/lovecraft-fiction/data

In [6]:


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')


# Specify the directory where you want to extract the files
extract_dir = "/content/drive/MyDrive/CEIA/NLP/HPL-ext"
# Relatos
os.listdir("/content/drive/MyDrive/CEIA/NLP/HPL-ext")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Old_Bugs.txt',
 'Polaris.txt',
 'The_Case_of_Charles_Dexter_Ward.txt',
 'The_Call_of_Cthulhu.txt',
 'The_Cats_of_Ulthar.txt',
 'PickmanÔÇÖs_Model.txt',
 'The_Diary_of_Alonzo_Typer.txt',
 'Out_of_the_Aeons.txt',
 'The_Shunned_House.txt',
 'The_Strange_High_House_in_the_Mist.txt',
 'The_Silver_Key.txt',
 'The_Alchemist.txt',
 'The_Secret_Cave_or_John_Lees_Adventure.txt',
 'Sweet_Ermengarde.txt',
 'The_Battle_that_Ended_the_Century.txt',
 'The_Book.txt',
 'Poetry_and_the_Gods.txt',
 'The_Street.txt',
 'The_Statement_of_Randolph_Carter.txt',
 'The_Beast_in_the_Cave.txt',
 'The_Shadow_over_Innsmouth.txt',
 'The_Mystery_of_the_Grave-Yard.txt',
 'The_Outsider.txt',
 'The_Other_Gods.txt',
 'The_Nameless_City.txt',
 'The_Mysterious_Ship.txt',
 'The_Shadow_out_of_Time.txt',
 'The_Slaying_of_the_Monster.txt',
 'The_Rats_in_the_Walls.txt',
 'The_Night_Ocean.txt',
 'The_Quest_of_Iranon.txt',
 'Through_the_Gates_of_the_Silver_Key.txt',
 'The_Unnamable.txt',
 'concat.txt',
 'The_Terrible_Old_Man

In [7]:

import pandas as pd
import os

# Elijo mis favoritas aunque pueda generar un sesgo
filenames = ['The_Call_of_Cthulhu.txt', 'The_Shadow_over_Innsmouth.txt',
             'The_Colour_out_of_Space.txt', 'Beyond_the_Wall_of_Sleep.txt',
             'Herbert_WestÔÇöReanimator.txt', 'The_Shadow_out_of_Time.txt',
             'The_Thing_on_the_Doorstep.txt', 'The_Dunwich_Horror.txt']

# Specify the directory where the files are located
directory = "/content/drive/MyDrive/CEIA/NLP/HPL-ext"

# Create an empty list to store the lines from the files
all_lines = []

# Iterate through the filenames and read the content of each file
for filename in filenames:
  filepath = os.path.join(directory, filename)
  try:
    with open(filepath, 'r', encoding='utf-8') as file:
      lines = file.readlines()
      all_lines.extend(lines)  # Append all lines from the current file to the list
  except FileNotFoundError:
    print(f"File not found: {filename}")
  except Exception as e:
    print(f"Error reading {filename}: {e}")


# Create a pandas Series from the list of lines
df = pd.DataFrame({'lines':all_lines})
print(df.head(10))


                                               lines
0                              The Call of Cthulhu\n
1                                                 \n
2  (Found Among the Papers of the LateFrancis Way...
3  “Of such great powers or beings there may be c...
4  a survival of a hugely remote period when . . ...
5  perhaps, in shapes and forms long since withdr...
6  forms of which poetry and legend alone have ca...
7   mythical beings of all sorts and kinds. . . .”\n
8                             —Algernon Blackwood.\n
9                            I.The Horror in Clay.\n


### 1 - Preprocesamiento

In [8]:
!pip install tensorflow

In [9]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence


sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-9-d5120ec64863>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [10]:
# Demos un vistazo
sentence_tokens[:2]

[['the', 'call', 'of', 'cthulhu'], []]

### 2 - Crear los vectores (word2vec)

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [12]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [13]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [14]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 8251


In [15]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2931


### 3 - Entrenar embeddings

In [16]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1012853.375
Loss after epoch 1: 669342.5
Loss after epoch 2: 622701.375
Loss after epoch 3: 565996.75
Loss after epoch 4: 564232.25
Loss after epoch 5: 563635.75
Loss after epoch 6: 542090.5
Loss after epoch 7: 528813.5
Loss after epoch 8: 527649.5
Loss after epoch 9: 522788.5
Loss after epoch 10: 518825.5
Loss after epoch 11: 516815.5
Loss after epoch 12: 511573.5
Loss after epoch 13: 510198.0
Loss after epoch 14: 498158.5
Loss after epoch 15: 491365.0
Loss after epoch 16: 487253.0
Loss after epoch 17: 488023.0
Loss after epoch 18: 485716.0
Loss after epoch 19: 484771.0


(1462440, 2430060)

### 4 - Ensayar

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["monstrous"], topn=10)

[('daemoniac', 0.8331003785133362),
 ('carrying', 0.8281660079956055),
 ('inconceivable', 0.827540934085846),
 ('unholy', 0.8185465931892395),
 ('realms', 0.8177081942558289),
 ('reptilian', 0.8107852339744568),
 ('crouching', 0.8075696229934692),
 ('blasphemous', 0.8071035742759705),
 ('similar', 0.8061019778251648),
 ('loathsome', 0.8055006861686707)]

tiene muchas referencias religiosas

In [30]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["fear"], topn=10)

[('we', -0.12986738979816437),
 ('old', -0.18095704913139343),
 ('toward', -0.2021089792251587),
 ('who', -0.20519478619098663),
 ('at', -0.2136247456073761),
 ('an’', -0.2254146784543991),
 ('no', -0.237948477268219),
 ('from', -0.24052567780017853),
 ('where', -0.24066706001758575),
 ('they', -0.24151332676410675)]

Creo que asocia el miedo a la soledad porque la palabra que esta mas lejos de "fear" es "we"

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["ominous"], topn=10)

[('opposite', 0.8974629044532776),
 ('trembling', 0.8804545998573303),
 ('departed', 0.8793109655380249),
 ('southward', 0.8773654103279114),
 ('bulging', 0.8756779432296753),
 ('stretched', 0.8755826354026794),
 ('starting', 0.8747203946113586),
 ('muttering', 0.8738757371902466),
 ('frantically', 0.8724966049194336),
 ('travels', 0.8718829154968262)]

Recordaba que le gustaba mucho usar la palabra "omnious" pero me llama la atención las palabras a las que se asocia

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["cthulhu"], topn=10)

[('race’s', 0.8999467492103577),
 ('swath', 0.8703106045722961),
 ('chimney', 0.8690612316131592),
 ('r’lyeh', 0.8640187978744507),
 ('central', 0.8464122414588928),
 ('shapeless', 0.8415716886520386),
 ('deal', 0.8400035500526428),
 ('storm', 0.8303804397583008),
 ('priest', 0.8233614563941956),
 ('iä', 0.8207603096961975)]

Me encanto este resultado

In [23]:
# el método `get_vector` permite obtener los vectores:
vector_dunwich = w2v_model.wv.get_vector("dunwich")
print(vector_dunwich)

[ 0.06723788 -0.06645113  0.15956624  0.06988897 -0.15806553 -0.31699094
 -0.03829481  0.34093815  0.12196173 -0.19291247 -0.03005726 -0.1592724
 -0.01204528 -0.00300066 -0.10568792 -0.00744291  0.08694628 -0.17323136
 -0.22705527 -0.19864337 -0.11147244  0.07008919  0.23937668  0.29136255
  0.13669279 -0.01994118 -0.20617075  0.06079997 -0.06242201 -0.16135141
  0.123757    0.17649405 -0.00367225 -0.06429721 -0.01458948  0.04297954
  0.06971525 -0.1490648  -0.12938419  0.11883048 -0.17219067  0.09394025
 -0.09392437 -0.17673945  0.13075523  0.20794465 -0.02159062  0.13309138
  0.04132361  0.3152182  -0.03217066 -0.0608518   0.16521794 -0.02530473
  0.05414985  0.05427018 -0.00642227  0.04462576  0.06662945  0.00241411
 -0.05567911 -0.10980941  0.11275364  0.20679465 -0.06826481  0.06290303
  0.1022507  -0.03124072  0.070761   -0.09962888  0.07632957  0.13044302
  0.06233967 -0.30553314 -0.00111315 -0.00893193 -0.00363909  0.05145708
  0.10883687  0.15086609 -0.01790506 -0.11747979  0.

In [24]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_dunwich)

[('dunwich', 1.0),
 ('authorities', 0.7918523550033569),
 ('science', 0.7872336506843567),
 ('supreme', 0.7832684516906738),
 ('correspondence', 0.7831420302391052),
 ('visitor', 0.7774217128753662),
 ('tightly', 0.7773541212081909),
 ('government', 0.7772454023361206),
 ('bureau', 0.77422034740448),
 ('daylight', 0.7709236145019531)]

In [25]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dunwich"], topn=10)

[('authorities', 0.7918523550033569),
 ('science', 0.7872336506843567),
 ('supreme', 0.7832684516906738),
 ('correspondence', 0.7831420302391052),
 ('visitor', 0.7774217128753662),
 ('tightly', 0.7773541212081909),
 ('government', 0.7772454619407654),
 ('bureau', 0.77422034740448),
 ('daylight', 0.7709236145019531),
 ('osborn’s', 0.7680399417877197)]

En el caso del "Horror de Dunwich" el eje son las ciencias ocultas y los que se oponen son un grupo de cientificos representado al orden gubernamental
así que tiene sentido las relaciones encontradas

### Busco una analogía dentro del texto

In [22]:

import numpy as np

# Assuming 'w2v_model' is already defined from the previous code

try:
  evil_vector = w2v_model.wv.get_vector("evil")
  good_vector = w2v_model.wv.get_vector("good")
  god_vector = w2v_model.wv.get_vector("god")
except KeyError as e:
  print(f"Error: Word '{e}' not found in the vocabulary. Skipping calculation.")
  exit()


result_vector = god_vector + (evil_vector - good_vector)

similar_words = w2v_model.wv.most_similar(positive=[result_vector], topn=10)
similar_words


[('evil', 0.7749531865119934),
 ('god', 0.6942228674888611),
 ('fur', 0.6652569770812988),
 ('cursed', 0.660415768623352),
 ('oh', 0.6589531302452087),
 ('gawd', 0.6536293625831604),
 ('address', 0.6523926258087158),
 ('immediately', 0.6514557003974915),
 ('comes', 0.6479724049568176),
 ('limitless', 0.6475804448127747)]

Sigue siendo los mas parecido "evil"  y "god", yo buscaba algo parecido a demonio o algo así.

In [76]:
w2v_model.wv.most_similar(positive=["devil"], topn=10)

[('monsters', 0.8319581747055054),
 ('sarten', 0.8290517926216125),
 ('cursed', 0.8250266909599304),
 ('towards', 0.8164858222007751),
 ('spell', 0.8151410222053528),
 ('mostly', 0.8119334578514099),
 ('cleared', 0.8115505576133728),
 ('sech', 0.8110347390174866),
 ('lavinny’s', 0.8100199699401855),
 ('araound', 0.8089156746864319)]

Lo que tienen en común es "cursed" lo cual es coherente con lo que se buscaba

### 5 - Visualizar agrupación de vectores

In [67]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [69]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=500
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [73]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.